In [18]:
%run scripts/helper.py
%run scripts/models.py

In [2]:
crowd_train = load_file('./data/train.csv/train.csv', index_col='id')

In [3]:
y = crowd_train.median_relevance.values

In [4]:
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import make_scorer

In [5]:
kappa_scorer = make_scorer(quadratic_weighted_kappa, greater_is_better = True)

In [6]:
X = tweak_text(crowd_train)

C:\Anaconda\lib\site-packages\bs4\__init__.py:176: UserWarning: "http://i104.photobucket.com/albums/m175/champions_on_display/wincraft2013/januaryb/65497012.jpg" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
C:\Anaconda\lib\site-packages\bs4\__init__.py:176: UserWarning: "http://i104.photobucket.com/albums/m175/champions_on_display/wincraft2013/januaryb/65516012.jpg" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL,

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler

In [14]:
tfv = TfidfVectorizer(min_df=3, max_df=500, max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 2), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

svd = TruncatedSVD(n_components=200, algorithm='randomized', n_iter=5, random_state=None, tol=0.0)
scl = StandardScaler(copy=True, with_mean=True, with_std=True)

In [15]:
X_tfv = tfv.fit_transform(X)
X_svd = svd.fit_transform(X_tfv)
X_scl = scl.fit_transform(X_svd)

In [16]:
from sklearn.svm import SVC

clf = SVC(C=10.0, kernel='rbf', degree=3, 
        gamma=0.0, coef0=0.0, shrinking=True, probability=False, 
        tol=0.001, cache_size=200, class_weight=None, 
        verbose=False, max_iter=-1, random_state=None)

In [17]:
scores = cross_val_score(clf, X_scl, y, scoring=kappa_scorer, n_jobs=1, cv=2)
print 'min score %0.4f, max score %0.4f and mean score %0.4f ' %(scores.min(), scores.max(), scores.mean())

min score 0.5302, max score 0.5406 and mean score 0.5354 


In [20]:
tweaked_model, tfv, svd, scl = build_non_linear_model(X, y)

In [21]:
crowd_test = load_file('./data/test.csv/test.csv', index_col='id')

In [22]:
Xtest = tweak_text(crowd_test)

C:\Anaconda\lib\site-packages\bs4\__init__.py:176: UserWarning: "http://i104.photobucket.com/albums/m175/champions_on_display/wincraft2013/januaryb/65527" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
C:\Anaconda\lib\site-packages\bs4\__init__.py:176: UserWarning: "http://i104.photobucket.com/albums/m175/champions_on_display/wincraft2013/januarya/14146012.jpg" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and fe

In [23]:
preds = non_linear_model_predictions(tweaked_model, tfv, svd, scl, Xtest)

In [24]:
make_submission(crowd_test.index.values.astype(int), preds, './model-submissions/non_linear_processed.csv')